# Определение стоимости автомобилей

**Описание проекта**

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Инструкция по выполнению проекта**

Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: скорость работы, точность результата.

1. Загрузите и подготовьте данные.
1. Обучите разные модели. Для каждой попробуйте различные гиперпараметры.
1. Проанализируйте скорость работы и качество моделей.

Примечания:
* Для оценки качества моделей применяйте метрику RMSE.
* Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.
* Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найдите её.
* Поскольку модель градиентного бустинга может обучаться долго, измените у неё только два-три параметра.

**Описание данных**

Признаки:
* `DateCrawled` — дата скачивания анкеты из базы

* `VehicleType` — тип автомобильного кузова
* `RegistrationYear` — год регистрации автомобиля
* `Gearbox` — тип коробки передач
* `Power` — мощность (л. с.)
* `Model` — модель автомобиля
* `Kilometer` — пробег (км)
* `RegistrationMonth` — месяц регистрации автомобиля
* `FuelType` — тип топлива:
    * `Gasoline` – это английское слово, обозначающее автомобильное топливо, бензин.
    * `Petrol` (Petroleum) – название торговой марки, то же, что и Gasoline.
    * `LPG` (Liquified Petroleum Gas) – сжиженный газ (пропан-бутан). Газ, полученный при добыче и переработке нефти. В жидкое состояние переводят при охлаждении до критической температуры и последующей конденсации в результате отвода теплоты парообразования.
    * `CNG` (Compressed Natural Gas) – сжатый природный газ (метан): газообразные углеводороды, образующиеся в земной коре, высокоэкономичное энергетическое топливо.
* `Brand` — марка автомобиля
* `NotRepaired` — была машина в ремонте или нет
* `DateCreated` — дата создания анкеты
* `NumberOfPictures` — количество фотографий автомобиля
* `PostalCode` — почтовый индекс владельца анкеты (пользователя)
* `LastSeen` — дата последней активности пользователя

Целевой признак:
* `Price` — цена (евро)

## Подготовка данных

Импортируем библиотеки.

In [1]:
import pandas as pd
import lightgbm as lgb

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

print("Версия LightGBM: ", lgb.__version__)

Версия LightGBM:  2.3.1


Импортируем данные.

In [2]:
try:  # local import
    df = pd.read_csv('./datasets/autos.csv')
except:  # from Praktikum server
    df = pd.read_csv('/datasets/autos.csv')

Посмотрим на данные в первом приближении.

In [3]:
display(df.head())
df.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

**Признаки с пропусками**

**Признаки на удаление**

Вероятно, с этими признаками есть простор для feature engineering, однако у меня не возникло идей как их можно использовать.

* DateCrawled — дата
* RegistrationMonth — месяц регистрации, содержит нули
* DateCreated — дата
* NumberOfPictures — заполнен нулями
* LastSeen — дата

<div class="alert alert-block alert-info">
    
**Комментарий**

Признаки отсмотрены с помощью sns.violinplot(), hist(), unique() и sample(). Для визуальной чистоты оставлен только блок с выводами.
    
</div>

**Резюме**

Выберем максимально щадящую стратегию. Чем больше данных останется, тем лучше.

* Price — содержит 10,7к строк с нулевой ценой и еще 2,5k строк с ценой до 100 евро, для обучения такие объекты не подходят.
* VehicleType — заполним пропуски индикаторным значением.
* RegistrationYear — содержит цифры > 2016 и > 2019;
> кроме того есть годы регистрации < 1980, беглый осмотр показывает что там перемешаны раритетные автомобили со случайно введенными значениями; так как у нас нет цели предсказания цен на раритетные авто, а к тому же их количества может быть и недостаточно для точного предсказания, то разумно на этом этапе удалить такие объекты.
* Gearbox — заполним мажоритарным классом (manual).
* Power — содержит в одном столбце и мощность в л.с. и объем двигателя, 40к строк с нулями;
> микролитражные автомобили имеют объем от 700 см^2, возьмем 600 с запасом, таких объектов всего 394 поэтому ими можно пренебречь; нулевые строки удалим исходя из того, что если в данных есть объекты для восстановления, значит их должно хватить и для обучения модели;
* Model — шумный признак, содержит пропуски, и категории _klasse, _reihe, other; заполним пропуски индикаторным значением.
* FuelType — следует объединить категории petrol/gasoline, это одно и то же; пропуски заполним мажоритарным классом, так как машин на альтернативном топливе мало и цена ошибки не высока.
* Brand — содержит категорию "sonstige_autos" (нем. другие), которая несет с собой много шума, избавимся от таких объектов.
* NotRepaired — не очевидный признак, заполним пропуски индикаторным значением.
* PostalCode — содержит коды длины 5 и 4. Уберем избыточность и сократим коды до первых 3 (1) цифр.
> Удаление или модификация столбца на RidgeCV не даёт различий. На градиентном бустинге снижение вариативности даёт небольшой положительный эффект.

Почистим данные.

In [4]:
# Drop
select = ['DateCrawled', 'RegistrationMonth',
          'DateCreated', 'LastSeen', 'NumberOfPictures']
df = df.drop(columns=select)

In [5]:
# Slice
df = df[(df.Price >= 100) &
        (df.RegistrationYear >= 1980) & (df.RegistrationYear < 2020) &
        (df.Power > 0) & (df.Power < 600) &
        (df.Brand != 'sonstige_autos')
        ]

# Replace
df.FuelType.replace(to_replace='petrol', value='gasoline', inplace=True)

# Fill
df.VehicleType.fillna('n/a', inplace=True)
df.Gearbox.fillna('manual', inplace=True)
df.Model.fillna('n/a', inplace=True)
df.FuelType.fillna('gasoline', inplace=True)
df.NotRepaired.fillna('n/a', inplace=True)

# Alternative way (RMSE Test Score: 2512 против 2415 на RidgeCV)
# df.dropna(subset=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired'])

# Variability reduction
df.PostalCode -= df.PostalCode % 1000

print(df.shape)

(301910, 11)


## Обучение моделей

Заведём функцию для оценки RMSE.

In [6]:
def evaluate(estimator):
    print('Test  RMSE Score: %.0f'%mse(y_test, estimator.predict(X_test), squared=False))
    print('Train RMSE Score: %.0f'%mse(y_train, estimator.predict(X_train), squared=False), '\n')

Заведём класс для хранения результатов.

In [7]:
class Vault:
    def __init__(self):
        self.vault = dict(model=[],
                          rmse_test=[], rmse_train=[],
                          fit_time=[], predict_time=[],
                          parameters=[])

    def save(self, model, rmse_test, rmse_train, fit_time, predict_time, parameters):
        self.vault['model'].append(model)
        self.vault['rmse_test'].append(rmse_test)
        self.vault['rmse_train'].append(rmse_train)
        self.vault['fit_time'].append(fit_time)
        self.vault['predict_time'].append(predict_time)
        self.vault['parameters'].append(parameters)
        
    def pop(self):
        self.vault['model'].pop()
        self.vault['rmse_test'].pop()
        self.vault['rmse_train'].pop()
        self.vault['fit_time'].pop()
        self.vault['predict_time'].pop()
        self.vault['parameters'].pop()

    def display(self):
        return display(pd.DataFrame(self.vault).sort_values(by='rmse_test'))

In [8]:
stats = Vault()

Выделим категориальные признаки.

In [9]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
target = 'Price'

print('Категориальные признаки:', categorical)

Категориальные признаки: ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']


Кодируем категориальные признаки и подготовим выборки для линейных моделей.

In [10]:
df_encoded = pd.get_dummies(df, columns=categorical, drop_first=True)

X = df_encoded.drop(target, axis=1)
y = df_encoded[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### DummyRegressor

In [11]:
stats.save('DummyRegressor', 4558, 4567, '1.16 ms', '6.7 ms', 'strategy="mean"')

### RidgeCV

In [12]:
stats.save('RidgeCV', 2415, 2434, '16.4 s', '1.81 s', 'alphas=[1e-3, 1e-2, 1e-1, 1]')

### LassoCV

In [13]:
stats.save('LassoCV', 2415, 2434, '2min 37s', '1.79 s', 'alphas=[1e-3, 1e-2, 1e-1, 1]')

### LightGBM

Кодируем категориальные признаки и подготовим выборки для градиентного бустинга.

In [14]:
df_labeled = df.copy()

for cat in categorical:
    df_labeled[cat] = df_labeled[cat].astype('category').cat.codes

X = df_labeled.drop(target, axis=1)
y = df_labeled[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

train_dataset = lgb.Dataset(X_train, y_train, feature_name=X.columns.tolist(), categorical_feature=categorical)
test_dataset = lgb.Dataset(X_test, y_test, feature_name=X.columns.tolist(), categorical_feature=categorical)

**I. значения по-умолчанию: метод gbdt, 100 деревьев**

In [15]:
stats.save('LightGBM', 1565, 1542, '1.36 s', '1.5 s', 'gbdt, num_boost_round=100')

**II. метод gbdt, 1000 деревьев (переобучение)**

In [16]:
stats.save('LightGBM', 1475, 1307, '8.13 s', '10.7 s', 'gbdt, num_boost_round=1000')

**III. метод dart, 1000 деревьев**

In [17]:
stats.save('LightGBM', 1491, 1428, '4min 53s', '17 s', 'dart, num_boost_round=1000')

## Анализ моделей

In [18]:
stats.display()

,model,rmse_test,rmse_train,fit_time,predict_time,parameters
4,LightGBM,1475,1307,8.13 s,10.7 s,"gbdt, num_boost_round=1000"
5,LightGBM,1491,1428,4min 53s,17 s,"dart, num_boost_round=1000"
3,LightGBM,1565,1542,1.36 s,1.5 s,"gbdt, num_boost_round=100"
1,RidgeCV,2415,2434,16.4 s,1.81 s,"alphas=[1e-3, 1e-2, 1e-1, 1]"
2,LassoCV,2415,2434,2min 37s,1.79 s,"alphas=[1e-3, 1e-2, 1e-1, 1]"
0,DummyRegressor,4558,4567,1.16 ms,6.7 ms,"strategy=""mean"""


Для сравнения с градиентным бустингом я взял легковесные и быстрые линейные модели с L1 и L2 регуляризацией. Чтобы обозначить порог полностью случайного предсказания добавил Dummy Regressor.

Лучший результат в эксперименте показал градиентный бустинг. Метод демонстрирует существенный прирост качества уже с настройками по-умолчанию. Как и всегда, немаловажную роль играет очистка и подготовка данных.

Перебор параметров из документации фреймворка не дал значимого прироста качества и в основном приводил к переобучению. В тетради оставлены 3 самых интересных, на мой взгляд, прохода.